<a href="https://colab.research.google.com/github/shazzad-hasan/practice-deep-learning-with-pytorch/blob/main/variational_autoencoder/conv_vae_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import required libraries
import torch
import torchvision

import os
import random
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# check if cuda is available
train_on_gpu = torch.cuda.is_available()

if train_on_gpu:
  print("CUDA is available!")
else:
  print("CUDA is not available!")

device = torch.device('cuda') if train_on_gpu else torch.device('cpu')

In [ ]:
# set up the general configeration
def set_all_seeds(seed):
  os.environ["PL_GLOBAL_SEED"] = str(seed)
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed_all(seed)

random_seed = 125
set_all_seeds(random_seed)

In [ ]:
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

# convert data to torch.FloatTensor
transform = transforms.ToTensor()
# load dataset from torchvision.datasets module
train_data = datasets.MNIST(root="./data", train=True, download=True, transform=transform)
test_data = datasets.MNIST(root="./data", train=False, download=True, transform=transform)

# number of subprocess to use for data loading
num_workers = 0
# how many samples per batch to load
batch_size = 128
# prepare data loaders
train_loader = DataLoader(train_data, batch_size=batch_size, num_workers=num_workers)
test_loader = DataLoader(test_data, batch_size=batch_size, num_workers=num_workers)

In [ ]:
# print out some data stats
print("Number of training images: ", len(train_data))
print("Number of test images: ", len(test_data))

for inputs, _ in train_loader:
  print("Image batch dimension: ", inputs.shape)
  break

In [ ]:
# visualize a batch of training data
def imshow(img):
  plt.imshow(np.squeeze(img), cmap="gray")

# obtain one batch of training images
dataiter = iter(train_loader)
inputs, targets = dataiter.next()
inputs = inputs.numpy()

# plot some images
fig = plt.figure(figsize=(10,4))
for idx in np.arange(10):
  ax = fig.add_subplot(2, 10/2, idx+1, xticks=[], yticks=[])
  imshow(inputs[idx])
  ax.set_title(str(targets[idx].item()))

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class VAE(nn.Module):
  def __init__(self, in_channels, latent_dim):
    super(VAE, self).__init__()

    # Initialize conv layers and fully-connected layers for the encoder
    self.enConv1 = nn.Conv2d(in_channels, 16, padding=5)
    self.enConv2 = nn.Conv2d(16, 32, 5)
    self.enFC1 = nn.Linear(20*20*32, latent_dim)
    self.enFC2 = nn.Linear(20*20*32, latent_dim)

    # Initialize conv and fully-connected layers for the decoder
    self.deFC1 = nn.Linear(latent_dim, 20*20*32)
    self.deConv1 = nn.ConvTranspose2d(32, 16, 5)
    self.deConv2 = nn.ConvTranspose2d(16, in_channels, 5)

  def encoder(self, x):
    x = F.relu(self, enConv1(x))
    x = F.relu(self, enConv2(x))
    x = x.view(-1, 20*20*32)
    mu = self.enFC1(x)
    log_var = self.enFC2(x)
    return mu, log_var

  def reparameterize(self, mu, log_var):
    std = torch.exp(0.5*log_var)
    eps = torch.randn_like(std)
    sample = mu + std * eps
    return sample

  def decoder(self, z):
    x = F.relu(self.defc1(z))
    x = x.view(-1, 20*20*32)
    x = F.relu(self.deConv1(x))
    x = x = torch.sigmoid(self.deConv2(x))

  def forward(self, x):
    mu, log_var = self.encoder(x)
    z = self.reparameterize(mu, log_var)
    recon_x = self.decoder(z)
    return recon_x, mu, log_var

latent_dim = 
model = VAE()
model.to(device)
print(model)